In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
local_weights_file = 'pretrain_model_weights/inception_v3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

In [44]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [45]:
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation = 'sigmoid')(x)
model = Model( pre_trained_model.input, x) 
model.compile(optimizer = 'rmsprop', 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [46]:
TRAINING_DIR = "data/cats_vs_dogs/training"
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,batch_size=10,class_mode="binary",target_size = (150,150))

VALIDATION_DIR = "data/cats_vs_dogs/testing"
validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,batch_size=2,class_mode="binary",target_size = (150,150))

Found 2700 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [47]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 25,
            epochs = 5,
            validation_steps = 10,
            verbose = 1)

Epoch 1/5
25/25 [==============================] - 22s 879ms/step - loss: 3.9752 - accuracy: 0.6960 - val_loss: 0.4446 - val_accuracy: 0.8500
Epoch 2/5
25/25 [==============================] - 23s 910ms/step - loss: 0.8523 - accuracy: 0.8240 - val_loss: 0.0954 - val_accuracy: 0.9500
Epoch 3/5
25/25 [==============================] - 23s 901ms/step - loss: 0.6172 - accuracy: 0.8520 - val_loss: 0.0574 - val_accuracy: 0.9500
Epoch 4/5
25/25 [==============================] - 22s 872ms/step - loss: 0.5685 - accuracy: 0.8800 - val_loss: 0.0094 - val_accuracy: 1.0000
Epoch 5/5
25/25 [==============================] - 24s 961ms/step - loss: 0.8255 - accuracy: 0.8600 - val_loss: 0.8403 - val_accuracy: 0.9000
